<b> Importing Libraries </b>

In [1]:
import sqlite3
import random
from faker import Faker
from datetime import datetime, timedelta

<b> Creating connection and generating Faker Object.</b>

In [2]:
fake = Faker()
conn = sqlite3.connect("badge_login.db")
cursor = conn.cursor()


<b> Creating Departments table</b>
<p>Departments table would contain 2 columns department_id and department_name. Using a for loop and cursor object to insert the given values</p>

In [3]:
departments = [
    ("D01", "Accounting"),
    ("D02", "IT"),
    ("D03", "HR"),
    ("D04", "Construction")
]
for dept_id, dept_name in departments:
    cursor.execute("""
    INSERT OR IGNORE INTO dim_department (department_id, department_name)
    VALUES (?, ?)
    """, (dept_id, dept_name))


In [4]:
# creating department primary key
cursor.execute("SELECT department_key, department_id FROM dim_department")
dept_map = {dept_id: dept_key for dept_key, dept_id in cursor.fetchall()}


<b>Creating Employees table</b>
<p>Employees table contains multiple columns which indicates an employees'id, full name,job title,employment,pay rate and hire date</p>

In [5]:
employees = []
employment_types = ["Full-Time", "Part-Time"]

for i in range(1, 6):  # 5 employees
    dept_id = random.choice(list(dept_map.keys()))
    employees.append((
        f"E{i:03}",                       # employee_id
        fake.name(),                      # full_name
        dept_map[dept_id],                # department_key
        random.choice(["Analyst", "Manager", "Technician", "Worker"]),  # job_title
        random.choice(employment_types),  # employment_type
        round(random.uniform(15, 50), 2), # hourly_rate
        fake.date_between(start_date='-5y', end_date='today').isoformat(), # hire_date
        1                                 # is_active
    ))

cursor.executemany("""
INSERT OR IGNORE INTO dim_employee 
(employee_id, full_name, department_key, job_title, employment_type, hourly_rate, hire_date, is_active)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
""", employees)


In [6]:
# ---------------------------
# 3️⃣ Insert dummy timecards
# ---------------------------
cursor.execute("SELECT employee_key, hourly_rate FROM dim_employee")
employee_data = cursor.fetchall()

for emp_key, rate in employee_data:
    # Generate 2 random shifts per employee
    for _ in range(2):
        work_date = fake.date_between(start_date='-30d', end_date='today').isoformat()
        # Random shift between 4 and 8 hours
        hours = random.choice([4, 6, 8])
        time_in_hour = random.randint(6, 14)  # shift start between 6am-2pm
        time_in = f"{time_in_hour:02}:00"
        time_out_hour = time_in_hour + hours
        time_out = f"{time_out_hour:02}:00"
        gross_pay = round(hours * rate, 2)
        
        cursor.execute("""
        INSERT INTO fact_timecards
        (employee_key, work_date, time_in, time_out, hours_worked, gross_pay)
        VALUES (?, ?, ?, ?, ?, ?)
        """, (emp_key, work_date, time_in, time_out, hours, gross_pay))

# Commit changes and close
conn.commit()

<b>Showing tables generated </b>

In [7]:
cursor.execute("SELECT * FROM dim_employee")
rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 'E001', 'Carly Day', 3, 'Manager', 'Part-Time', 45.68, '2025-11-21', 1)
(2, 'E002', 'Alexander Craig', 4, 'Technician', 'Full-Time', 23.39, '2023-03-20', 1)
(3, 'E003', 'James Lloyd', 4, 'Manager', 'Part-Time', 29.67, '2024-01-26', 1)
(4, 'E004', 'Chad King', 2, 'Analyst', 'Part-Time', 31.21, '2023-08-22', 1)
(5, 'E005', 'Julia Hernandez', 2, 'Technician', 'Part-Time', 24.08, '2025-12-22', 1)


In [8]:
cursor.execute("SELECT * FROM dim_department")
rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 'D01', 'Accounting')
(2, 'D02', 'IT')
(3, 'D03', 'HR')
(4, 'D04', 'Construction')


In [9]:
cursor.execute("SELECT * FROM fact_timecards")
rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 1, '2026-01-21', '14:00', '20:00', 6.0, 274.08)
(2, 1, '2026-01-09', '11:00', '19:00', 8.0, 365.44)
(3, 2, '2026-01-10', '06:00', '10:00', 4.0, 93.56)
(4, 2, '2026-01-18', '09:00', '17:00', 8.0, 187.12)
(5, 3, '2026-01-10', '12:00', '18:00', 6.0, 178.02)
(6, 3, '2026-01-09', '13:00', '21:00', 8.0, 237.36)
(7, 4, '2026-01-05', '11:00', '17:00', 6.0, 187.26)
(8, 4, '2026-01-02', '07:00', '13:00', 6.0, 187.26)
(9, 5, '2026-01-05', '13:00', '17:00', 4.0, 96.32)
(10, 5, '2026-01-07', '12:00', '16:00', 4.0, 96.32)
(11, 1, '2025-12-31', '10:00', '14:00', 4.0, 182.72)
(12, 1, '2026-01-09', '08:00', '12:00', 4.0, 182.72)
(13, 2, '2026-01-05', '12:00', '18:00', 6.0, 140.34)
(14, 2, '2026-01-19', '10:00', '14:00', 4.0, 93.56)
(15, 3, '2026-01-07', '07:00', '13:00', 6.0, 178.02)
(16, 3, '2025-12-28', '14:00', '20:00', 6.0, 178.02)
(17, 4, '2026-01-04', '10:00', '16:00', 6.0, 187.26)
(18, 4, '2026-01-01', '06:00', '10:00', 4.0, 124.84)
(19, 5, '2026-01-03', '08:00', '14:00', 6.0, 144.48)
(20, 5

<b>Show specific employee with ID = E001" </b>


In [10]:
cursor.execute("""
SELECT
    e.employee_id,
    e.full_name,
    t.work_date,
    t.time_in,
    t.time_out,
    t.hours_worked
FROM fact_timecards t
JOIN dim_employee e
    ON t.employee_key = e.employee_key
WHERE e.employee_id = 'E001'
ORDER BY t.work_date
""")

rows = cursor.fetchall()

for row in rows:
    print(row)

('E001', 'Carly Day', '2025-12-29', '08:00', '16:00', 8.0)
('E001', 'Carly Day', '2025-12-31', '10:00', '14:00', 4.0)
('E001', 'Carly Day', '2026-01-01', '07:00', '11:00', 4.0)
('E001', 'Carly Day', '2026-01-01', '14:00', '20:00', 6.0)
('E001', 'Carly Day', '2026-01-02', '10:00', '18:00', 8.0)
('E001', 'Carly Day', '2026-01-02', '11:00', '15:00', 4.0)
('E001', 'Carly Day', '2026-01-03', '14:00', '18:00', 4.0)
('E001', 'Carly Day', '2026-01-05', '08:00', '12:00', 4.0)
('E001', 'Carly Day', '2026-01-09', '11:00', '19:00', 8.0)
('E001', 'Carly Day', '2026-01-09', '08:00', '12:00', 4.0)
('E001', 'Carly Day', '2026-01-15', '09:00', '15:00', 6.0)
('E001', 'Carly Day', '2026-01-17', '14:00', '18:00', 4.0)
('E001', 'Carly Day', '2026-01-19', '08:00', '16:00', 8.0)
('E001', 'Carly Day', '2026-01-20', '09:00', '13:00', 4.0)
('E001', 'Carly Day', '2026-01-21', '14:00', '20:00', 6.0)
('E001', 'Carly Day', '2026-01-24', '13:00', '21:00', 8.0)
('E001', 'Carly Day', '2026-01-24', '08:00', '12:00', 4.

In [11]:
conn.commit()
conn.close()